In [157]:
from constraint import *
import re as re

In [80]:
models = []
mission_types = {}
pilots = {}
missions = {}
nuas = []

def parse_inputs():
    f = open('inputs.txt')
    inputs = f.readlines()
    f.close()
    for i in range(len(inputs)):
        line = inputs[i]
        if not line[0] == "%":
            line = line.rstrip()
#             print(line)
            if line[0:2] == "MT":
                line = line.replace("  "," ")
                line = line.split(" ")  
                mission_type = line[1]
                compat_string = list(line[2])
                compat = [int(i) for i in compat_string]            
                mission_types[mission_type] = compat
            elif line[0:2] == "P ":
                line = line.replace("  "," ")
                line = line.split(" ")
                pilot = line[1]
                compat_string = list(line[3])
                compat = [int(i) for i in compat_string]            
                pilots[pilot] = compat
            elif line[0] == "M" and i > 14:
                line = line.replace("  "," ")
                line = line.replace("M ", "")
                line = line.replace("M", "")
                line = line.split(" ")
                mission_num = line[0]
                mission_type = line[1][0]
                compat = mission_types[mission_type]
                missions[mission_num] = compat
            elif "NUAS" in line:
                line = line.replace("  "," ")
                line = line.split(" ")
                l_nuas = list(line[1])
                l_nuas = [int(i) for i in l_nuas]
                for i in l_nuas:
                    nuas.append(i)
            else:   
                models.append(line)
    print("Number of missions:", len(missions.keys()))
    print("Number of unique models:", len(models))
    

In [81]:
parse_inputs()

# number of missions
n_missions = len(missions.keys())

# number of pilots
n_pilots = len(pilots.keys())

#number of unique models
n_unique_models = len(models)

Number of missions: 20
Number of unique models: 9


In [110]:
nuas

[1, 1, 1, 4, 1, 2, 5, 1, 1]

In [187]:
problem = Problem()

pilot_model_variables = []
for model in range(n_unique_models):
    for pilot in range(n_pilots):
        var = "p"+str(pilot)+"_m"+str(model)+"_"
        problem.addVariable(var, [0,1])
        pilot_model_variables.append(var)

mission_model_variables = []
for type in range(n_missions):
    for model in range(n_unique_models):
        var = "t"+str(type)+"_m"+str(model)+"_"
        problem.addVariable(var, [0,1])
        mission_model_variables.append(var)


In [188]:
# add constraints

#each pilot drives only uav:
for pilot in range(n_pilots):
    current_pilot = "p"+str(pilot)+"_"
    current_pilot_vars = [s for s in pilot_model_variables if current_pilot in s]
    problem.addConstraint(MaxSumConstraint(1), current_pilot_vars)

#pilot model compatibility constraint
pilot_keys = list(pilots.keys())
for pilot in range(n_pilots):
    compatible_models = pilots[pilot_keys[pilot]]
    for model in range(n_unique_models):
        var = "p"+str(pilot)+"_m"+str(model)+"_"
        compatibility = compatible_models[model]
        problem.addConstraint(lambda x: x <= compatibility, [var])
        
    
#Number of models constraint?
for model in range(n_unique_models):
    current_model = "m"+str(model)+"_"
    current_model_vars = [s for s in pilot_model_variables if current_model in s]
    num_available = nuas[model]
    if len(current_model_vars) > 0:
        problem.addConstraint(MaxSumConstraint(num_available), current_model_vars)
        
# one mission one UAV
for type in range(n_missions):
    current_mission = "t"+str(type)+"_"
    current_mission_vars = [s for s in mission_model_variables if current_mission in s]
    problem.addConstraint(ExactSumConstraint(1), current_mission_vars)

    

#mission model compatibility constaint
mission_keys = list(missions.keys())
for type in range(n_missions):
    compatible_models = missions[mission_keys[type]]
    for model in range(n_unique_models):
        var = "t"+str(type)+"_m"+str(model)+"_"
        compatibility = compatible_models[model]
        problem.addConstraint(lambda x: x <= compatibility, [var])
        
# each uav can be used 3 times max
for model in range(n_unique_models):
    var = "m"+str(model)+"_"
    num_models = nuas[model]
    current_model_missions = [s for s in mission_model_variables if var in s]
    current_model_pilots = [s for s in pilot_model_variables if var in s]
    problem.addConstraint(MaxSumConstraint(3), current_model_missions)


In [189]:
solution = problem.getSolution()

In [203]:
print(nuas)
sol_keys = list(solution.keys())
keys = [key for key in sol_keys if "m4_" in key ]
for key in keys:
    print(key+":"+str(solution[key]))

[1, 1, 1, 4, 1, 2, 5, 1, 1]
p0_m4_:0
p1_m4_:0
p2_m4_:0
p3_m4_:0
t0_m4_:0
t10_m4_:0
t11_m4_:0
t12_m4_:0
t13_m4_:0
t14_m4_:0
t15_m4_:0
t16_m4_:0
t17_m4_:0
t18_m4_:0
t19_m4_:0
t1_m4_:0
t2_m4_:1
t3_m4_:1
t4_m4_:1
t5_m4_:0
t6_m4_:0
t7_m4_:0
t8_m4_:0
t9_m4_:0


In [205]:
mission_keys = list(missions.keys())
# solution_keys = list(solution.keys())
for mission in range(n_missions):
    current_mission = None
    current_model = None
    current_pilot = None
    current_mission = mission_keys[mission]
    
    current_mission_model_variables = [i for i in mission_model_variables if "t"+str(mission)+"_m" in i]
    for var in current_mission_model_variables:
        if solution[var] == 1:
            model_num = re.findall(r'\d+', var)[1]
            current_model = model_num + "_" + models[int(model_num)]
            current_pilot_model_variables = [i for i in pilot_model_variables if "m"+model_num+"_" in i]
#             print(current_pilot_model_variables)
            for pilot_var in current_pilot_model_variables:
#                 print(pilot_var, solution[pilot_var])
                if solution[pilot_var] == 1:
                    current_pilot_num = re.findall(r'\d+', pilot_var)[0]
                    current_pilot = list(pilots.keys())[int(current_pilot_num)]
    print("Current Mission:", current_mission)
    print("Current Model:", current_model)
    print("Current Pilot:", current_pilot)
    print("\n")

        

Current Mission: 1
Current Model: 0_AR200
Current Pilot: Merrick


Current Mission: 2
Current Model: 3_Mavic
Current Pilot: Major


Current Mission: 3
Current Model: 4_M600
Current Pilot: None


Current Mission: 4
Current Model: 4_M600
Current Pilot: None


Current Mission: 5
Current Model: 4_M600
Current Pilot: None


Current Mission: 6
Current Model: 5_P4
Current Pilot: None


Current Mission: 7
Current Model: 5_P4
Current Pilot: None


Current Mission: 8
Current Model: 5_P4
Current Pilot: None


Current Mission: 9
Current Model: 6_PHK
Current Pilot: None


Current Mission: 10
Current Model: 6_PHK
Current Pilot: None


Current Mission: 11
Current Model: 0_AR200
Current Pilot: Merrick


Current Mission: 12
Current Model: 0_AR200
Current Pilot: Merrick


Current Mission: 13
Current Model: 1_Disco
Current Pilot: Adams


Current Mission: 14
Current Model: 1_Disco
Current Pilot: Adams


Current Mission: 15
Current Model: 1_Disco
Current Pilot: Adams


Current Mission: 16
Current Model: 2_

In [192]:
pilots

{'Adams': [1, 1, 0, 1, 1, 1, 1, 0, 1],
 'Gingrich': [0, 0, 0, 1, 0, 0, 0, 0, 0],
 'Major': [0, 0, 0, 0, 0, 0, 0, 1, 0],
 'Merrick': [0, 0, 0, 1, 0, 0, 0, 0, 0]}